In [3]:
import seaborn as sns 
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd 

from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

In [4]:
Plant_Dataset = pd.read_csv('Feature_Extracted_Plant_Dataset.csv')
Plant_Dataset.describe()

,Edge_Count,Re(Mean_Frequency),Im(Mean_Frequency),Frequency_Variance,StDev_Contrast,Mean_Noise,StDev_Noise,Minimum_Intensity,Maximum_Intensity
count,1500.000000,1.500000e+03,1.500000e+03,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
mean,3733.535333,5.989650e-01,-5.655018e-19,22014.959039,0.130521,0.536565,0.130521,0.145936,0.956581
std,3703.635039,1.935199e-01,2.498024e-17,9643.229257,0.042781,0.170824,0.042781,0.128510,0.081043
min,0.000000,2.287667e-17,-1.023487e-16,767.239036,0.039815,0.032910,0.039815,0.000000,0.512022
25%,880.750000,4.726469e-01,-1.517883e-17,15588.933324,0.099020,0.471818,0.099020,0.042561,0.957370
50%,2208.000000,6.035386e-01,-3.252607e-19,22965.973814,0.123368,0.574923,0.123368,0.112293,0.994167
75%,5652.250000,7.168188e-01,1.507041e-17,29156.339680,0.155001,0.654635,0.155001,0.225440,1.000000
max,19455.000000,1.000000e+00,1.179612e-16,45719.011444,0.329416,0.832021,0.329416,0.594200,1.000000
